In [17]:
!pip install zenml


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('data/olist_customers_dataset.csv')

In [6]:
data.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'review_score',
       'review_comment_message'],
      dtype='object')

In [28]:
import logging as log
import numpy as np
from typing import Union
from abc import ABC , abstractmethod
import pandas as pd
from sklearn.model_selection import train_test_split

class DataStrategy(ABC):
        @abstractmethod
        def handle_data(self , data: pd.DataFrame) -> Union[pd.DataFrame , pd.Series]:
                pass

class DataPreProcessingStrategy(DataStrategy):
        def handle_data(self, data: pd.DataFrame) -> Union[pd.DataFrame, pd.Series]:
            try:
                log.info(f"data Columns While Pre processing {data.columns}")

                data =  data.drop([
                    "order_approved_at" ,
                    "order_delivered_carrier_date" ,
                    "order_delivered_customer_date" ,
                    "order_estimated_delivery_date",
                    "order_purchase_timestamp" ,
                ],axis=1)

                data["product_weight_g"].fillna(data["product_weight_g"].median(), inplace=True)
                data["product_length_cm"].fillna(data["product_length_cm"].median(), inplace=True)
                data["product_height_cm"].fillna(data["product_height_cm"].median(), inplace=True)
                data["product_width_cm"].fillna(data["product_width_cm"].median(), inplace=True)
                
                data["review_comment_message"].fillna("No review", inplace=True)

                data = data.select_dtypes(include=[np.number])

                cols_to_drop = ["customer_zip_code_prefix" , "order_item_id"]

                data = data.drop(cols_to_drop , axis=1)
                return data
            except Exception as e:
                log.error(f"Error in Pre processing the data {e}")
                raise e

class DataSplitStrategy(DataStrategy):
      def handle_data(self, data: pd.DataFrame) -> Union[pd.DataFrame, pd.Series]:
        try:
            log.info(f"Data Columns {data}")
            X = data.drop(["review_score"] , axis=1)
            y = data['review_score']
            X_train , X_test , y_train , y_test = train_test_split(X,y , test_size=0.2 , random_state=42)
            return X_train , X_test , y_train , y_test
        
        except Exception as e:
            log.error(f"Error while Data Splitting {e}")
            raise e


class DataCleaning:
        def __init__(self,data: pd.DataFrame , strategy: DataStrategy):
          self.data = data
          self.strategy = strategy

        def handle_data(self) -> Union[pd.DataFrame, pd.Series]:
            try:
                return self.strategy.handle_data(self.data)
            except Exception as e:
                log.error("Error while Data Splitting: {}".format(e))
                raise e
                    



In [31]:
from pipelines.train import train_pipeline

@pipeline 
def train_pipeline(data_path: str):
#     ingested_data = ingest_data(data_path)
    pass
    # log.info(f"Ingested data: {ingested_data.columns}")
    # X_train, X_test, y_train, y_test = clean_data(ingested_data)
#     processed_data = clean_data(ingested_data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/40/j2m2x4q57nn09vx26gkdv6h40000gn/T/ipykernel_41673/3142926027.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/40/j2m2x4q57nn09vx26gkdv6h40000gn/T/ipykernel_41673/3142926027.py'                 │
│                                                                                                  │
│ /Users/macbook/Desktop/HOME/Ai_Projects/mlOps/customer_satisfaction/pipelines/train.py:2 in      │
│ <module>                                                                                         │
│                                                                                                  │
│    1 from zenml import pipeline                                                                  │
│ ❱  2 from steps.ingest_data import ingest_data                                                   │
│    3 from steps.clean_data import clean_data                                                     │
│    4 from steps.model_train import train_model                                                   │
│    5 from steps.evaluation import evaluate                                                       │
│                                                                                                  │
│ /Users/macbook/Desktop/HOME/Ai_Projects/mlOps/customer_satisfaction/steps/ingest_data.py:28 in   │
│ <module>                                                                                         │
│                                                                                                  │
│   25                                                                                             │
│   26                                                                                             │
│   27 @step                                                                                       │
│ ❱ 28 def ingest_data(data_path: str) -> pd.DataFrame:                                            │
│   29 │   """                                                                                     │
│   30 │   this will ingest the data from data_path                                                │
│   31                                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.9/site-packages/zenml/new/steps/step_decorator.py:155 in step             │
│                                                                                                  │
│   152 │   if _func is None:                                                                      │
│   153 │   │   return inner_decorator                                                             │
│   154 │   else:                                                                                  │
│ ❱ 155 │   │   return inner_decorator(_func)                                                      │
│   156                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/site-packages/zenml/new/steps/step_decorator.py:123 in inner_decorator  │
│                                                                                                  │
│   120 │   """                                                                                    │
│   121 │                                                                                          │
│   122 │   def inner_decorator(func: "F") -> "BaseStep":                                          │
│ ❱ 123 │   │   from zenml.new.steps.decorated_step import _D

In [ ]:
pip install "zenml[server]"

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 482.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 1.5 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached starlette-0.27.0-py3-none-any.whl (66 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 520.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 1.8 MB/s eta 0:00:0000:0100:01
  Using cached h11-0.14.0-py3-none-any

  Using cached httptools-0.6.0-cp39-cp39-macosx_10_9_x86_64.whl (166 kB)
  Using cached websockets-11.0.3-cp39-cp39-macosx_10_9_x86_64.whl (120 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached uvloop-0.17.0-cp39-cp39-macosx_10_9_x86_64.whl (1.5 MB)


  Created wheel for orjson: filename=orjson-3.8.14-cp39-cp39-macosx_10_7_x86_64.whl size=305386 sha256=9c1c5c52ba13fa19840c36f061f42160d1da9a72f63044193f64be64ce1554cb
  Stored in directory: /Users/macbook/Library/Caches/pip/wheels/7b/48/ca/e698c5ac33f5b1ae5baeaab3f71ba94005fd4fe9fc44dbb9e6
Successfully built orjson
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebr

  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://g